In [ ]:
!pip install --quiet langchain
!pip install --quiet crewai crewai-tools
!pip install --quiet litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.3/537.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.3 MB/s 

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from datetime import datetime
import requests
import json
from litellm import completion
import os

from google.colab import userdata
from langchain.tools import tool

In [ ]:
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
)


In [ ]:
class LinkedInPostGeneratorAgent():
    def web_search_agent(self):
        return Agent(
            role='Web Search Agent',
            goal='Search the internet for relevant and recent information about the given topic.',
            backstory='As a digital researcher, you find accurate and relevant information to support the given topic, ensuring high-quality input for LinkedIn posts.',
            tools=[SearchTool.search_internet],
            verbose=True,
            allow_delegation=False,  # Focused on searching only
            asyn_execution=True,  # Web search can be performed asynchronously
            llm=llm
        )

    def storytelling_enhancer_agent(self):
        return Agent(
            role='Storytelling Enhancer',
            goal='Transform the search results and input topic into engaging, narrative-driven LinkedIn posts.',
            backstory='You use storytelling techniques to make posts more relatable and memorable, ensuring a strong connection with the audience.',
            verbose=True,
            allow_delegation=False,  # Focused solely on enhancing narratives
            asyn_execution=False,  # Operates inline with the content flow
            llm=llm
        )

    def post_compiler_agent(self):
        return Agent(
            role="Post Compiler",
            goal="Compile the outputs from all tasks into a cohesive, final LinkedIn post ready for publishing.",
            backstory="As the final step in the process, you integrate all refined content, humanized narratives, and optimized hashtags into a polished LinkedIn post.",
            verbose=True,
            allow_delegation=False,  # Final step; no delegation needed
            asyn_execution=False,  # Operates synchronously to finalize the post
            llm=llm
        )


In [ ]:
class LinkedInPostTasks():
    def web_search_task(self, agent, topic):
        return Task(
            description=f"Search the internet for the latest and most relevant information about the topic: {topic}",
            agent=agent,
            async_execution=True,
            expected_output="""A summary of the top 3-5 search results with titles, URLs, and snippets.
            Example Output:
            [
                {'title': 'The Rise of AI in Healthcare',
                 'url': 'https://example.com/article1',
                 'snippet': 'AI is transforming the healthcare industry...',
                 'date': '2023-01-15'},
                {...}
            ]"""
        )

    def storytelling_task(self, agent, context):
        return Task(
            description="Transform the search results and given topic into a compelling LinkedIn post draft with a strong narrative.",
            agent=agent,
            context=context,
            expected_output="""A narrative-driven LinkedIn post draft.
            Example Output:
            'AI is transforming healthcare in remarkable ways. From predictive analytics to personalized medicine, here are three ways AI is revolutionizing the industry...'
            """
        )

    def compile_post_task(self, agent, context):
        return Task(
            description="Compile the outputs from all previous tasks into a cohesive and polished LinkedIn post ready for publishing.",
            agent=agent,
            context=context,
            expected_output="""A complete LinkedIn post in the following format:
            Example Output:
            **Title:** AI Transforming Healthcare
            **Body:** AI is transforming healthcare in remarkable ways. From predictive analytics to personalized medicine, here are three ways AI is revolutionizing the industry.
            **Key Takeaways:**
            1. Predictive analytics improves early disease detection.
            2. Personalized medicine tailors treatments to individual patients.
            3. Operational efficiencies reduce costs and improve outcomes.
            **Hashtags:** #AI #HealthcareInnovation #TechTrends
            """
        )

In [ ]:
serper_api_key = userdata.get('SERPER_API_KEY')

class SearchTool():
  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet about a given topic and return relevant results"""
    print("Searching the internet")
    top_result_to_return = 3
    url = "https://google.serper.dev/search"
    payload = json.dumps(
        {
            "q": query,
            "num": top_result_to_return,
            "tbm": "nws"
        }
    )
    headers = {
        'X-API-KEY': serper_api_key,
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    if 'organic' not in response.json():
      return "Sorry I couldn't find anything about that, there could be an error \
      with your serper api key."
    else:
      results = response.json()['organic']
      string = []
      print(f"Results: {results[:top_result_to_return]}")
      for result in results[:top_result_to_return]:
        try:
          data = result.get('date', 'Date not available')
          string.append('\n'.join([
                        f'Title: {result["title"]}',
                        f'Link: {result["link"]}',
                        f'Snippet: {result["snippet"]}',
                        f'Date: {data}',
                        "\n----------------------------",
                    ]))
        except:
          print("error")
      return '\n'.join(string)

In [ ]:
# prompt: give me example topic related to AI technology to use to create linkedin post

topic_ideas = [
    "The ethical implications of artificial intelligence in healthcare",
    "The future of work in the age of automation",
    "The role of AI in combating climate change",
    "The impact of generative AI on creative industries",
    "The potential of AI to personalize education",
    "The use of AI in cybersecurity and its challenges",
    "The latest advancements in natural language processing and their applications",
    "How AI is transforming customer service and support",
    "The importance of data privacy and security in the age of AI",
    "The growing field of AI-powered drug discovery and development"
]

import random

# Choose a random topic from the list
random_topic = random.choice(topic_ideas)

print(f"Topic for LinkedIn post: {random_topic}")

In [ ]:
agents = LinkedInPostGeneratorAgent()
tasks = LinkedInPostTasks()

web_search = agents.web_search_agent()
storytelling_enhancer = agents.storytelling_enhancer_agent()
post_compiler = agents.post_compiler_agent()


topic = "How DeepSeek threatend the rule of OpenAI"  # Example topic provided by the user
web_search_task = tasks.web_search_task(agent=web_search, topic=topic)
storytelling_task = tasks.storytelling_task(storytelling_enhancer, [web_search_task])
compile_post_task = tasks.compile_post_task(post_compiler, [storytelling_task])

crew = Crew(
    agents=[web_search, storytelling_enhancer, post_compiler],
    tasks=[web_search_task, storytelling_task, compile_post_task],
    process=Process.hierarchical,
    manager_llm=llm,
    verbose=True,
)

results = crew.kickoff()
print(f"Crew work results: {results}")

LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object
LLM value is a string
# Agent: Crew Manager
## Task: Search the internet for the latest and most relevant information about the topic: How DeepSeek threatend the rule of OpenAI
# Agent: Web Search Agent
## Task: Search the internet for the latest and most relevant information about the topic: How DeepSeek threatened the rule of OpenAI. Return a JSON array of objects, where each object contains the title, URL, a short snippet, and ideally, the publication date of the top 3-5 search results.
Searching the internet
Results: [{'title': 'A shocking Chinese AI advancement called DeepSeek is ... - CNN', 'link': 'https://www.cnn.com/2025/01/27/tech/deepseek-stocks-ai-china/index.html', 'snippet': '“Time will tell if the DeepSeek threat is real — the race is on as to what technology works and how the big Western players will respond and ...', 'date': '10 hours ago', 'position': 1}, {'title':

In [ ]:
from IPython.display import Markdown

def display_markdown(markdown_text):
  """
  Displays markdown text as markdown in a Jupyter environment.
  """
  display(Markdown(markdown_text))

display_markdown(results.raw)

**Title:** DeepSeek Challenges OpenAI's Dominance in the AI Race

**Body:** The global AI landscape is experiencing a seismic shift with the emergence of DeepSeek, a Chinese AI company rapidly challenging OpenAI's established leadership.  DeepSeek's innovative advancements are sparking discussions about a potential reshuffling of market share and the future trajectory of the AI industry.  Experts are closely analyzing DeepSeek's approach and its potential to reshape the competitive dynamics of the global AI race.

**Key Takeaways:**
1. DeepSeek's rapid advancements present a significant challenge to OpenAI's dominance.
2. The competition between DeepSeek and OpenAI could accelerate innovation in the AI sector.
3. This development highlights the increasingly global and competitive nature of the AI industry.

**Hashtags:** #AI #DeepSeek #OpenAI #ArtificialIntelligence #Tech #China #Innovation #AIRevolution #TechCompetition
```